In [82]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [84]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-06 06:00:00+00:00


In [85]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [86]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-06 00:45:33,618 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 00:45:33,620 INFO: Initializing external client
2025-03-06 00:45:33,621 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 00:45:35,092 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215676


In [87]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.58s) 


In [88]:
df['pickup_hour']

0     2025-03-06 05:00:00+00:00
1     2025-03-06 05:00:00+00:00
2     2025-03-06 05:00:00+00:00
3     2025-03-06 05:00:00+00:00
4     2025-03-06 05:00:00+00:00
                 ...           
247   2025-03-06 05:00:00+00:00
248   2025-03-06 05:00:00+00:00
249   2025-03-06 05:00:00+00:00
250   2025-03-06 05:00:00+00:00
251   2025-03-06 05:00:00+00:00
Name: pickup_hour, Length: 252, dtype: datetime64[us, Etc/UTC]

In [89]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-06 00:45:59,438 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 00:45:59,440 INFO: Initializing external client
2025-03-06 00:45:59,440 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 00:46:00,781 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215676
Current UTC time: 2025-03-06 05:45:59.438060+00:00
Next hour: 2025-03-06 06:00:00+00:00
Querying for date range: 2025-03-06 to 2025-03-07
Filtering for hour: 2025-03-06 06:00
Found 0 records


In [90]:
now = datetime.now(timezone.utc)

In [91]:
predictions

,pickup_location_id,predicted_demand,pickup_hour


In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  252 non-null    int32                  
 1   predicted_demand    252 non-null    float64                
 2   pickup_hour         252 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 5.1 KB


In [93]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [94]:
dt = current_date.ceil('h')


In [95]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-06 00:46:13,586 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 00:46:13,588 INFO: Initializing external client
2025-03-06 00:46:13,588 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 00:46:15,038 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215676
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.54s) 


In [96]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.54s) 


,pickup_location_id,predicted_demand,pickup_hour


In [97]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,51,1.0,2025-03-06 05:00:00+00:00
1,151,36.0,2025-03-06 05:00:00+00:00
2,16,0.0,2025-03-06 05:00:00+00:00
3,28,0.0,2025-03-06 05:00:00+00:00
4,191,0.0,2025-03-06 05:00:00+00:00
...,...,...,...
247,25,2.0,2025-03-06 05:00:00+00:00
248,141,133.0,2025-03-06 05:00:00+00:00
249,198,0.0,2025-03-06 05:00:00+00:00
250,232,6.0,2025-03-06 05:00:00+00:00


In [98]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [99]:
predictions = fetch_next_hour_predictions()

2025-03-06 00:46:36,297 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 00:46:36,300 INFO: Initializing external client
2025-03-06 00:46:36,300 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 00:46:37,592 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215676
Current UTC time: 2025-03-06 05:46:36.297604+00:00
Next hour: 2025-03-06 06:00:00+00:00
Found 0 records


In [100]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-03-06 00:46:43,336 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 00:46:43,339 INFO: Initializing external client
2025-03-06 00:46:43,339 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 00:46:44,644 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215676


In [101]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.54s) 


In [102]:
df["pickup_hour"].max()

Timestamp('2025-03-06 05:00:00+0000', tz='Etc/UTC')

In [103]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-06 06:00:00+00:00


In [104]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
